In [ ]:
import hashlib
import datetime
import psycopg2

class Block:
    def __init__(self, index, previous_hash, timestamp, data):
        self.index = index
        self.previous_hash = previous_hash
        self.timestamp = timestamp
        self.data = data
        self.current_hash = self.calculate_hash()

    def calculate_hash(self):
        value = str(self.index) + self.previous_hash + str(self.timestamp) + str(self.data)
        return hashlib.sha256(value.encode('utf-8')).hexdigest()

class Blockchain:
    def __init__(self):
        self.conn = psycopg2.connect(
            dbname='your_database_name', user='your_username', password='your_password', host='localhost'
        )
        self.cur = self.conn.cursor()

        self.create_genesis_block()

    def create_genesis_block(self):
        timestamp = datetime.datetime.now()
        genesis_block = Block(0, "0", timestamp, "Genesis Block")

        self.cur.execute("""
            INSERT INTO blocks (index, previous_hash, timestamp, data, current_hash)
            VALUES (%s, %s, %s, %s, %s);
        """, (genesis_block.index, genesis_block.previous_hash, genesis_block.timestamp,
              genesis_block.data, genesis_block.current_hash))

        self.conn.commit()

    def get_latest_block(self):
        self.cur.execute("SELECT * FROM blocks ORDER BY index DESC LIMIT 1;")
        result = self.cur.fetchone()
        if result:
            index, previous_hash, timestamp, data, current_hash = result
            return Block(index, previous_hash, timestamp, data)
        else:
            return None

    def add_block(self, data):
        latest_block = self.get_latest_block()
        new_index = latest_block.index + 1
        timestamp = datetime.datetime.now()
        new_block = Block(new_index, latest_block.current_hash, timestamp, data)

        self.cur.execute("""
            INSERT INTO blocks (index, previous_hash, timestamp, data, current_hash)
            VALUES (%s, %s, %s, %s, %s);
        """, (new_block.index, new_block.previous_hash, new_block.timestamp,
              new_block.data, new_block.current_hash))

        self.conn.commit()

    def hash_data(self, data):
        return hashlib.sha256(data.encode('utf-8')).hexdigest()

    def add_hashed_data(self, nama, alamat, umur, nik):
        hashed_nama = self.hash_data(nama)
        hashed_alamat = self.hash_data(alamat)
        hashed_umur = self.hash_data(umur)
        hashed_nik = self.hash_data(nik)

        data_hashed = {
            "Nama": hashed_nama,
            "Alamat": hashed_alamat,
            "Umur": hashed_umur,
            "NIK": hashed_nik
        }

        data_diri = {
            hashed_nama: nama,
            hashed_alamat: alamat,
            hashed_umur: umur,
            hashed_nik: nik
        }

        self.cur.execute("""
            INSERT INTO data_store (hash_key, data)
            VALUES (%s, %s), (%s, %s), (%s, %s), (%s, %s);
        """, (hashed_nama, nama, hashed_alamat, alamat, hashed_umur, umur, hashed_nik, nik))

        self.conn.commit()

        self.add_block(data_hashed)
        print("Blok dengan data hash berhasil ditambahkan.")

    def get_original_data(self, hashed_key):
        self.cur.execute("SELECT data FROM data_store WHERE hash_key = %s;", (hashed_key,))
        result = self.cur.fetchone()
        if result:
            return result[0]
        else:
            return "Data tidak ditemukan"

    def display_chain(self):
        self.cur.execute("SELECT * FROM blocks ORDER BY index;")
        rows = self.cur.fetchall()
        for row in rows:
            index, previous_hash, timestamp, data, current_hash = row
            print(f"Index: {index}")
            print(f"Previous Hash: {previous_hash}")
            print(f"Timestamp: {timestamp}")
            print(f"Data: {data}")
            print(f"Hash: {current_hash}")
            print("--------------")

# Membuat instance dari Blockchain
my_blockchain = Blockchain()

# Loop untuk menambahkan blok
while True:
    print("Menu:")
    print("1. Tambah data baru")
    print("2. Tampilkan blockchain")
    print("3. Ambil data asli")
    print("4. Keluar")

    choice = input("Pilih opsi (1/2/3/4): ")

    if choice == '1':
        # Meminta input dari pengguna untuk data diri
        nama = input("Masukkan nama: ")
        alamat = input("Masukkan alamat: ")
        umur = input("Masukkan umur: ")
        nik = input("Masukkan NIK: ")

        # Menambahkan blok dengan data diri yang di-hash
        my_blockchain.add_hashed_data(nama, alamat, umur, nik)

    elif choice == '2':
        # Menampilkan blockchain
        my_blockchain.display_chain()

    elif choice == '3':
        # Meminta input dari pengguna untuk hash kunci
        print("Masukkan hash kunci untuk data yang ingin diambil:")
        print("1. Hash Nama")
        print("2. Hash Alamat")
        print("3. Hash Umur")
        print("4. Hash NIK")
        hashed_key_choice = input("Pilih opsi (1/2/3/4): ")
        if hashed_key_choice == '1':
            hashed_key = input("Masukkan hash Nama: ")
        elif hashed_key_choice == '2':
            hashed_key = input("Masukkan hash Alamat: ")
        elif hashed_key_choice == '3':
            hashed_key = input("Masukkan hash Umur: ")
        elif hashed_key_choice == '4':
            hashed_key = input("Masukkan hash NIK: ")
        else:
            print("Pilihan tidak valid.")
            continue

        original_data = my_blockchain.get_original_data(hashed_key)
        print(f"Data asli: {original_data}")

    elif choice == '4':
        # Keluar dari loop
        print("Keluar dari program.")
        break

    else:
        print("Pilihan tidak valid. Silakan pilih 1, 2, 3, atau 4.")

# Menutup koneksi saat program selesai
my_blockchain.cur.close()
my_blockchain.conn.close()
